# Test Scientist: Run an Experiment

In [1]:
from simulator.simulation.scientists import Scientist
from winds.generators import OscillatoryGenerator

import numpy as np
from numpy import pi

import requests

## Load or Create Winds

We'll use the HTTP interface to do this, issuing commands via the appropriate *url* endpoints...

### WindGenParams:

In [11]:
# wind velocity oscillating in XY-plane (Circle)
windgenparams = {
    'is_oscillatory': True,
    'base_velocity': np.array([0,0,0]),
    'amplitude': np.array([1,1,0]),
    'frequency': np.array([1,1,1]),
    'phase_offset': np.array([0,np.pi/2,0]),
}

res = requests.post('http://localhost:8000/winds/wind-gen-params/', data=windgenparams)
res_data = res.json()
if res.status_code == 409:
    message = res_data['message']
    print(message)
else:
    print('Created.')
obj_id = res_data['id']
print(obj_id)

Already exists
c6c7c0a8-0c92-4ef7-8d6f-76323ce287ae


### WindSpacetime:

In [12]:
windspacetimeparams = {
    'generator_name': 'oscillatory',
    'generator_params': obj_id,
    'duration': 100,
    'timestep': .01,
}

res = requests.post('http://localhost:8000/winds/wind-spacetimes/', data=windspacetimeparams)
res_data = res.json()
windspacetime_id = res_data['id']
print(res_data)
print(windspacetime_id)

{'message': 'Already exists', 'id': '0ce749e3-cae5-4575-ab1c-ee4b8ab47d54'}
0ce749e3-cae5-4575-ab1c-ee4b8ab47d54


Great, we now have the UUID for a windspacetime specified by the parameters above. We now pass this id as a parameter to the scientist used to run the experiment.

## Run Experiment

We'll call the scientist class directly until we setup an HTTP view to encapsulate this logic.

In [13]:
params = {
        'windspacetime_id': windspacetime_id,
        'num_trials': 10,
        'prob_speed_fn_name': 'Uniform',
        'max_initial_speed': 42, # m/s
        'prob_speed_center': None,
        'prob_speed_spread': None,
        'prob_timing_fn_name': 'Uniform',
        'max_wait_time': 1, # s
        'prob_timing_center': None,
        'prob_timing_spread': None,
        'prob_aiming_fn_name': 'Uniform',
        'prob_aiming_geometry': 'Spherical',
        'prob_aiming_X1_min': -pi/4,  # -45 deg
        'prob_aiming_X1_max': pi/4,   #  45 deg
        'prob_aiming_X1_center': None,
        'prob_aiming_X1_spread': None,
        'prob_aiming_X2_min': pi/6,   # 30 deg down from vertical 
        'prob_aiming_X2_max': 2*pi/3, # 120 deg down from vertical = 30 deg below horiztonal
        'prob_aiming_X2_center': None,
        'prob_aiming_X2_spread': None,
        'prob_aiming_X3_min': 0,      # throwaway coordinate
        'prob_aiming_X3_max': 1,
        'prob_aiming_X3_center': None,
        'prob_aiming_X3_spread': None,
        'timestep': 0.01,

}
scientist = Scientist(params)
scientist.run_experiment()

TypeError: Scientist.load_windspacetime() takes 1 positional argument but 2 were given